In [ ]:
## 미완성 비지표물 채권 무위험수익률 커브 그리기
## https://www.wsj.com/market-data/bonds/treasuries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import QuantLib as ql
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import urllib.request
import requests

options = webdriver.ChromeOptions()
options.add_argument('headless') # headless: 브라우저가 안뜸

driver = webdriver.Chrome('C:\\Users\\inventor\\Desktop\\FR study\\FR-30th\\Bond_study\\FR_30_NIH\\chromedriver.exe', options=options)

def GET_DATE():
    # url = "https://www.wsj.com/market-data/bonds"
    # req = requests.get(url,headers=headers)
    # html = req.content
    # soup = BeautifulSoup(html, 'html.parser')
    URL="https://www.wsj.com/market-data/bonds"
    driver.get(URL)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    data = soup.select('div > div > div > div > div > div > div > div > h3 > span')
    # print(data)
    date = data[1].text
    # print(date[-8:])
    if date[-8]=='1':
        date = datetime.datetime.strptime(str(date[-8:]), "%m/%d/%y")
    else:
        date = datetime.datetime.strptime(str(date[-7:]), "%m/%d/%y")
    print(date)
    return date
    
def GET_QUOTE(reference_date):
    
    URL="https://www.wsj.com/market-data/bonds/treasuries"
    driver.get(URL)

    source = driver.page_source
    tbonds_code = BeautifulSoup(source,'html.parser')

    #xpath을 이용한 방법
    button_xpath = '//*[@id="root"]/div/div/div/div[2]/div/div/div[3]/ul/li[2]/button'
    button = driver.find_element_by_xpath(button_xpath)
    button.click()

    source = driver.page_source
    tbills_code = BeautifulSoup(source,'html.parser')

    #date 가져오기 
    date = tbonds_code.select('div > div > div > div > div > div > div > div > h3 > span')
    data = date[1].text
    # print(data)

    #tbond table 가져오기
    tbond_rows = tbonds_code.select('div > div > div > div > div > div > div > div > table > tbody > tr')
    # print(tbond_rows)

    #tbill table 가져오기
    tbill_rows = tbills_code.select('div > div > div > div > div > div > div > div > table > tbody > tr')
    # print(tbill_rows)

    tbond_content = []
    tbond_contents = []

    for tbond_row in tbond_rows:
        tds = tbond_row.find_all("td")
        for td in tds:
            tbond_content.append(td.text)
        tbond_contents.append(tbond_content)
        tbond_content=[]

    tbond_df=pd.DataFrame(tbond_contents)
    tbond_headers=['Maturity', 'Coupon','Bid','Ask','Chg', 'Asked Yield']
    tbond_df.columns=tbond_headers
    tbond_df.set_index('Maturity',inplace=True)


    tbill_content = []
    tbill_contents = []

    for tbill_row in tbill_rows:
        tds = tbill_row.find_all("td")
        for td in tds:
            tbill_content.append(td.text)
        tbill_contents.append(tbill_content)
        tbill_content=[]

    tbill_df=pd.DataFrame(tbill_contents)
    tbill_headers=['Maturity','Bid','Ask','Chg', 'Asked Yield']
    tbill_df.columns=tbill_headers
    tbill_df.set_index('Maturity',inplace=True)
    

    
    # Create Empty Lists
    maturities = []
    days = []
    prices = []
    coupons = []

        
        # Price
        if i <= 3:
            data_src = soup.find("span", id="quote_val")
            price = data_src.text
            price = float(price[:-1])
        else:
            data_src = soup.find("span", id="price_quote_val")
            price = data_src.text
            price = price.split()
            price1 = float(price[0])
            price = price[1].split('/')
            price2 = float(price[0])
            price3 = float(price[1])            
            price = price1 + (price2 / price3)  
            
        data_src2 = soup.find_all("span", class_="data_data")
        
        # Coupon
        coupon = data_src2[2].text
        if coupon != '':
            coupon = float(coupon[:-1])
        else:
            coupon = 0.0
        
        # Maturity Date
        maturity = data_src2[3].text
        maturity = datetime.datetime.strptime(maturity, '%m/%d/%y')
        
        # Send to Lists
        days.append((maturity - reference_date).days)
        prices.append(price)
        coupons.append(coupon)
        maturities.append(maturity)
    
    # Create DataFrame
    df = pd.DataFrame([maturities, days, prices, coupons]).transpose()
    headers = ['maturity', 'days', 'price', 'coupon']
    df.columns = headers
    df.set_index('maturity', inplace=True)
    
    return df


def TREASURY_CURVE(date, quote1, quote2):
    
    # Divide Quotes
    tbill = quote1
    tbond = quote2
    
    # Set Evaluation Date
    eval_date = ql.Date(date.day, date.month, date.year)
    ql.Settings.instance().evaluationDate = eval_date
    
    # Set Market Conventions
    calendar = ql.UnitedStates()
    convention = ql.ModifiedFollowing
    day_counter = ql.ActualActual()
    end_of_month = True
    fixing_days = 1
    face_amount = 100
    coupon_frequency = ql.Period(ql.Semiannual)
    
    # Construct Treasury Bill Helpers
    bill_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                         ql.Period(m, ql.Days),
                                         fixing_days,
                                         calendar,
                                         convention,
                                         end_of_month,
                                         day_counter)
                    for r, m in zip(tbill['price'], tbill['days'])]
    
    print(bill_helpers)
    
    # Construct Treasury Bond Helpers
    bond_helpers = []
    for p, c, m in zip(tbond['price'], tbond['coupon'], tbond['days']):
        termination_date = eval_date + ql.Period(m, ql.Days)
        schedule = ql.Schedule(eval_date,
                               termination_date,
                               coupon_frequency,
                               calendar,
                               convention,
                               convention,
                               ql.DateGeneration.Backward,
                               end_of_month)
        bond_helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(p)),
                                             fixing_days,
                                             face_amount,
                                             schedule,
                                             [c/100.0],
                                             day_counter,
                                             convention)
        bond_helpers.append(bond_helper)
    
    print(bond_helpers)
    # Bind Helpers
    rate_helper = bill_helpers + bond_helpers
    print(rate_helper)
    
    # Build Curve
    yc_linearzero = ql.PiecewiseLinearZero(eval_date, rate_helper, day_counter)
    print(yc_linearzero)
    
    return yc_linearzero

def DISCOUNT_FACTOR(date, curve):
    date = ql.Date(date.day, date.month, date.year)
    return curve.discount(date)

def ZERO_RATE(date, curve):
    date = ql.Date(date.day, date.month, date.year)
    day_counter = ql.ActualActual()
    compounding = ql.Compounded
    freq = ql.Continuous
    zero_rate = curve.zeroRate(date, day_counter, compounding, freq).rate()
    return zero_rate
